In [ ]:

import pysqlite3
import os
from datetime import datetime
from dotenv import load_dotenv
import libsql_experimental as libsql


load_dotenv()

True

In [6]:
""" Connect to Turso - embedded replicas """

DB_URL = os.getenv("TURSO_DATABASE_URL")
AUTH_TOKEN = os.getenv("TURSO_AUTH_TOKEN")
OPENAI_SECRET = os.getenv("OPEN_AI_SECRET")

conn = libsql.connect("../data/primary.db", sync_url=DB_URL, auth_token=AUTH_TOKEN)

conn.sync()

2024-12-03T05:44:37.707430Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:44:37.767434Z  WARN libsql::replication::remote_client: Frames prefetching failed because of new session token returned by handshake


In [7]:
indicator_tables = [
    # broad
    "real_gdp",
    "cpi",
    "treasury_yields",
    "federal_funds_rate",
    "unemployment_rate",
    "inflation_rate",
    
    # Forex
    "forex_usd_aud",
    "forex_aud_jpy",

    # Commodities
    "commodity_wti",
    "commodity_brent",
    "commodity_copper",
    "commodity_aluminum",
]

In [10]:
from openai import OpenAI

def generate_macro_summary(indicators):
    """
    Generate a summary of the macroeconomic environment using OpenAI.
    :param indicators: Dictionary containing economic indicator data.
    :return: Markdown summary as a string.
    """
    prompt = f"""
    Generate a concise and insightful summary of the current macroeconomic environment based on the following data:
    {indicators}

    Please explain the trends and their potential implications for the economy in markdown format.
    """
    
    client = OpenAI(api_key=OPENAI_SECRET)
    
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a professional economic advisor and analyst."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return completion.choices[0].message

def create_summary_table():


    conn.execute("""
    CREATE TABLE IF NOT EXISTS macro_summaries (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created_at TEXT,
        summary TEXT
    )
    """)
    conn.commit()
    conn.sync()
    
    
def save_summary_to_db(summary):
    """
    Save a generated summary into the SQLite database.
    :param summary: The markdown-formatted summary string.
    :param db_path: Path to SQLite database.
    """
    # conn = pysqlite3.connect(DB_PATH)
    # cursor = conn.cursor()

    # Ensure the summary table exists
    conn.execute("""
    CREATE TABLE IF NOT EXISTS macro_summaries (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created_at TEXT,
        summary TEXT
    )
    """)

    # Insert the new summary
    conn.execute("""
    INSERT INTO macro_summaries (created_at, summary) 
    VALUES (?, ?)
    """, (datetime.now().isoformat(), summary))

    conn.commit()
    conn.sync()

def fetch_all_indicators(tables):
    """
    Fetch data from all specified tables and aggregate it into a dictionary.
    :param tables: List of table names to query.
    :param db_path: Path to SQLite database.
    :return: Dictionary with table names as keys and data as lists of records.
    """


    indicators = {}
    for table in tables:
        if table.startswith("forex"):  # Handle forex tables with OHLC data
            rows = conn.execute(f"SELECT date, open, high, low, close FROM {table} ORDER BY date DESC LIMIT 30").fetchall()
            
            
            indicators[table] = [
                {
                    "date": row[0],
                    "open": row[1],
                    "high": row[2],
                    "low": row[3],
                    "close": row[4]
                }
                for row in rows
            ]
        else:  # Handle standard value-based tables
            rows = conn.execute(f"SELECT date, value FROM {table} ORDER BY date DESC LIMIT 30").fetchall()
            
            indicators[table] = [
                {"date": row[0], "value": row[1]}
                for row in rows
            ]
    conn.sync()
    return indicators

def format_indicators_for_prompt(indicators):
    """
    Format indicators dictionary into a string for the OpenAI prompt.
    :param indicators: Dictionary of indicator data.
    :return: Formatted string.
    """
    formatted = []
    for category, data in indicators.items():
        formatted.append(f"### {category.replace('_', ' ').title()}")
        for record in data:
            if "value" in record:  # Standard tables
                formatted.append(f"- {record['date']}: {record['value']}")
            else:  # Forex tables
                formatted.append(
                    f"- {record['date']}: Open={record['open']}, High={record['high']}, "
                    f"Low={record['low']}, Close={record['close']}"
                )
    return "\n".join(formatted)
    


In [11]:
# Step 1: Ensure summary table exists
create_summary_table()

# Step 2: Fetch economic indicators
db_indicators = fetch_all_indicators(indicator_tables)

# Format the indicators for the OpenAI prompt
formatted_indicators = format_indicators_for_prompt(db_indicators)

# Step 3: Generate summary
summary = generate_macro_summary(formatted_indicators)

# # Step 4: Save summary to the database
save_summary_to_db(summary.content)

2024-12-03T05:46:07.697806Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:07.736855Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:07.794713Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:16.501539Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:16.551870Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:16.611460Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:16.650918Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.
2024-12-03T05:46:16.675163Z  INFO libsql::replication::remote_client: Attempting to perform handshake with primary.


In [18]:
def fetch_latest_summary():
    # conn = pysqlite3.connect(DB_PATH)
    # cursor = conn.cursor()

    row = conn.execute("""
    SELECT created_at, summary FROM macro_summaries
    ORDER BY created_at DESC
    LIMIT 1
    """).fetchone()
    
    # conn.sync()

    return {"created_at": row[0], "summary": row[1]} if row else None

In [19]:
from IPython.display import Markdown, display

# Fetch the latest summary
latest_summary = fetch_latest_summary()

if latest_summary:
    print(f"Summary created at: {latest_summary['created_at']}")
    display(Markdown(latest_summary["summary"]))
else:
    print("No summaries found in the database.")

Summary created at: 2024-12-03T16:46:16.536529


# Macroeconomic Environment Summary

## Economic Growth
- **Real GDP**: Real GDP displayed consistent growth from January 2023 to July 2024, peaking at 5866.67 in July 2024. This indicates robust economic activity and recovery post-pandemic. However, the slight dip in October 2023 suggests potential fluctuations in economic momentum.

## Inflation and Prices
- **Consumer Price Index (CPI)**: From January 2023 to October 2024, the CPI steadily increased from 299.17 to 315.664. This reflects ongoing inflationary pressures, albeit at a moderated pace compared to the sharp increases of previous years, largely due to base effects and potential supply chain improvements.
- **Inflation Rate**: The annual inflation rate decreased significantly from 8% in 2022 to approximately 4.1% by January 2023. Continued monitoring is essential, as persistent inflation could impact consumer spending and saving behaviors.

## Interest Rates and Yields
- **Treasury Yields**: Treasury yields have seen fluctuations, stabilizing around 4.36% by November 2024 after peaking at 4.8% in October 2023. This suggests cautious investor sentiment regarding future economic conditions and monetary policy shifts.
- **Federal Funds Rate**: The Federal Funds Rate remained relatively high at around 5.33% for most of 2024, indicating the central bank's focus on managing inflation. Recent reductions to 4.64% by November 2024 might reflect easing inflation concerns or efforts to stimulate economic growth.

## Labor Market
- **Unemployment Rate**: The unemployment rate slightly increased from around 3.4% in early 2023 to 4.1% by October 2024. Although still relatively low, this trend might indicate cooling in labor market tightness, which could affect wage growth dynamics and consumer spending power.

## Commodities and Forex
- **Oil Prices (WTI and Brent)**: Oil prices have shown significant volatility, with WTI and Brent experiencing peaks and falls. This is reflective of changing global supply-demand dynamics, geopolitical tensions, and production adjustments.
- **Copper and Aluminum**: Both commodities experienced price increases in 2024, reflecting strong demand possibly driven by industrial activity and technological advancements.
- **Forex (USD/AUD and AUD/JPY)**: The USD/AUD exchange rate showed volatility, which could affect trade balances and export competitiveness. Similarly, fluctuations in AUD/JPY hint at shifts in investor risk appetite and interest rate differentials.

## Implications
1. **Growth Sustainability**: The continuing growth in Real GDP, despite recent volatility, suggests that the economic recovery is ongoing. However, sustained inflation and interest rates could pose risks to future growth.
2. **Monetary Policy**: With inflation easing, central banks may consider rate adjustments to support growth without stoking inflation further, which requires careful balance.
3. **Consumer and Investor Confidence**: Fluctuations in commodity prices and forex rates indicate underlying market uncertainties, potentially affecting consumer and investor confidence levels.
4. **Labor Market Dynamics**: The uptick in unemployment suggests potential structural shifts or sector-specific impacts that could affect overall economic sentiment and policy decisions.

Continued vigilance is required to navigate these dynamics, ensuring balanced growth through adaptive monetary and fiscal policies.